In [404]:
"""
PAR Servicios Document Processing - Notebook Optimizado para Prompt Caching
MODIFICACIÓN: Procesa primero TODAS las clasificaciones, luego TODAS las extracciones
para maximizar el aprovechamiento del prompt caching.

Flujo optimizado:
1. FASE 1: Clasificación de todos los documentos (orden: ACC, CECRL, CERL, RUB, RUT)
2. FASE 2: Extracción de todos los documentos (mismo orden, agrupados por categoría)
"""

import os
import sys
import json
import logging
import shutil
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple
import boto3
from dataclasses import dataclass
import tempfile
from datetime import datetime
import io
from PyPDF2 import PdfReader, PdfWriter
from string import Template
import re
from botocore.config import Config
import base64
import pdfplumber
import time


In [405]:
# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuración del notebook
NOTEBOOK_ROOT = Path.cwd()
PROJECT_STRUCTURE_ROOT = NOTEBOOK_ROOT / "par_servicios_testing"
TEST_DOCUMENTS_ROOT = NOTEBOOK_ROOT / "test_documents"

# Variables de entorno para simular el proyecto
os.environ.setdefault("LAMBDA_TASK_ROOT", str(PROJECT_STRUCTURE_ROOT / "classification" / "src"))
os.environ.setdefault("BEDROCK_MODEL", "us.amazon.nova-pro-v1:0")
os.environ.setdefault("FALLBACK_MODEL", "us.anthropic.claude-sonnet-4-20250514-v1:0")
os.environ.setdefault("REGION", "us-east-2")
os.environ.setdefault("AWS_PROFILE", "par_servicios")
CATEGORIES_ORDER = ["ACC", "CECRL", "CERL", "RUB", "RUT"]
config = Config(read_timeout=1000)

print("🚀 PAR Servicios Testing Notebook - Estructura Real del Proyecto")
print("="*70)

🚀 PAR Servicios Testing Notebook - Estructura Real del Proyecto


In [406]:
def create_bedrock_client(region, profile):
    """
    Create Bedrock clients for the specified region
    
    Args:
        region (str): AWS region (e.g., 'us-east-1')
        
    Returns:
        tuple: (bedrock, bedrock_runtime) clients
    """
    # Initialize Bedrock clients
    session = boto3.Session(profile_name=profile)
    bedrock_adm = session.client(
        service_name='bedrock',
        region_name=region,
        config=config
    )
    
    bedrock_client = session.client(
        service_name='bedrock-runtime',
        region_name=region,
        config=config
    )
    
    return bedrock_adm, bedrock_client

In [407]:
bedrock_adm, bedrock_client = create_bedrock_client(os.environ["REGION"],os.environ["AWS_PROFILE"])

2025-09-02 10:27:37,492 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [408]:
def sanitize_name(raw_name: str) -> str:
    """
    Strip off any disallowed characters (including periods) from a filename.
    We'll just keep the stem (no extension) and remove anything but
    alphanumerics, spaces, hyphens, parentheses, and brackets.
    """
    from pathlib import Path
    stem = Path(raw_name).stem  # e.g. "244_CA_2020-02-29"
    # Remove underscores and periods, keep only allowed chars
    safe = "".join(ch for ch in stem
                   if ch.isalnum()
                   or ch in " -()[]")
    # Collapse multiple spaces or hyphens if you like
    return safe or "document"  # fallback if everything got stripped

In [409]:
def clean_json_text(text: str) -> str:
    """Clean text - simpler approach"""
    # Remove control characters only
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return text
    

In [410]:
def _strip_fences(text: str) -> str:
    """
    Removes ```json ... ``` or ``` ... ``` even if the opening fence is
    immediately followed by '{'.
    """
    text = text.strip()
    # opening fence
    text = re.sub(r'^```(?:json)?', '', text, flags=re.IGNORECASE).lstrip()
    # closing fence
    text = re.sub(r'```$', '', text).rstrip()
    text = clean_json_text(text)
    return text

In [411]:
def _normalise(raw_obj: dict, *, file_path: str | None = None) -> dict:
    """
    • rename documenttype → document_type (case-insensitive)
    • ensure document_number and path exist (fallbacks)
    • keep snippet only if present
    """
    norm = {k.lower(): v for k, v in raw_obj.items()}  # case-fold keys

    # key mapping
    if "documenttype" in norm and "document_type" not in norm:
        norm["document_type"] = norm.pop("documenttype")

    # fallback values
    if "document_number" not in norm:
        # try to derive from the file path    e.g.  .../800035887/...
        if file_path:
            m = re.search(r"/(\d{6,})/", file_path)
            norm["document_number"] = m.group(1) if m else "UNKNOWN"
        else:
            norm["document_number"] = "UNKNOWN"

    if "path" not in norm and file_path:
        norm["path"] = file_path
    elif "path" not in norm:
        norm["path"] = "UNKNOWN"

    return norm

In [412]:
def clean_text_for_json(text):
    """
    Clean text to remove unusual line terminators and other problematic characters
    that can break JSON parsing and file handling.
    """
    if not isinstance(text, str):
        return text
    
    import unicodedata
    
    # Remove unusual line terminators
    text = text.replace('\u2028', ' ')  # Line Separator (LS)
    text = text.replace('\u2029', ' ')  # Paragraph Separator (PS)
    text = text.replace('\u000B', ' ')  # Vertical Tab
    text = text.replace('\u000C', ' ')  # Form Feed
    text = text.replace('\u0085', ' ')  # Next Line (NEL)
    
    # Remove other control characters that can cause issues
    text = ''.join(char for char in text if unicodedata.category(char) not in ['Cc', 'Cf'] or char in ['\n', '\r', '\t'])
    
    # Normalize whitespace
    text = ' '.join(text.split())
    
    return text

In [413]:
def extract_pdf_text_with_pdfplumber(pdf_bytes: bytes) -> str:
    """
    Extrae texto del PDF usando pdfplumber como fallback cuando el PDF es muy grande.
    """
    try:
        text_content = ""
        
        with pdfplumber.open(io.BytesIO(pdf_bytes)) as pdf:
            logger.info(f"Extrayendo texto con pdfplumber - {len(pdf.pages)} páginas")
            
            for i, page in enumerate(pdf.pages, 1):
                page_text = page.extract_text()
                if page_text:
                    text_content += f"--- PÁGINA {i} ---\n"
                    text_content += page_text + "\n\n"
        
        # Limpiar el texto extraído
        text_content = clean_text_for_json(text_content.strip())
        
        logger.info(f"Texto extraído exitosamente: {len(text_content)} caracteres")
        return text_content
        
    except Exception as e:
        logger.error(f"Error extrayendo texto con pdfplumber: {e}")
        return f"[ERROR EXTRAYENDO TEXTO: {str(e)}]"

In [414]:
def extract_pdf_text_with_textract(pdf_bytes: bytes, region: str = None, profile: str = None) -> str:
    """
    Extrae texto del PDF usando AWS Textract - FLUJO CORREGIDO
    """
    try:
        # Usar la región y perfil configurados o los por defecto
        region = os.environ.get("REGION", "us-east-2")
        profile = os.environ.get("AWS_PROFILE", "par_servicios")
        
        # Crear cliente de Textract
        session = boto3.Session(profile_name=profile)
        textract_client = session.client(
            service_name='textract',
            region_name=region,
            config=config
        )
        
        logger.info(f"Extrayendo texto con AWS Textract - PDF size: {len(pdf_bytes)} bytes")
        
        # Llamar a Textract para detectar texto (tu archivo hardcodeado)
        response = textract_client.start_document_text_detection(
            DocumentLocation={
                'S3Object': {
                    'Bucket': "applying-par-textract",
                    'Name': "890900608.pdf"
                }
            }
        )
        
        job_id = response['JobId']
        logger.info(f"Job iniciado: {job_id}")

        # PASO 1: ESPERAR A QUE EL JOB TERMINE
        max_wait_time = 300  # 5 minutos máximo
        wait_interval = 10   # Revisar cada 10 segundos
        elapsed_time = 0
        
        while elapsed_time < max_wait_time:
            logger.info(f"Verificando estado del job... ({elapsed_time}s transcurridos)")
                
            job_response = textract_client.get_document_text_detection(JobId=job_id)
            job_status = job_response['JobStatus']
                
            if job_status == 'SUCCEEDED':
                logger.info("Job completado exitosamente!")
                break
            elif job_status == 'FAILED':
                error_msg = job_response.get('StatusMessage', 'Unknown error')
                raise RuntimeError(f"Textract job failed: {error_msg}")
            elif job_status in ['IN_PROGRESS', 'PARTIAL_SUCCESS']:
                logger.info(f"Job status: {job_status}, esperando...")
                time.sleep(wait_interval)
                elapsed_time += wait_interval
            else:
                raise RuntimeError(f"Unexpected job status: {job_status}")
        
        # Verificar timeout FUERA del loop
        if elapsed_time >= max_wait_time:
            raise RuntimeError("Textract job timed out after 5 minutes")
        
        # PASO 2: OBTENER RESULTADOS (SOLO DESPUÉS DE QUE TERMINE)
        logger.info("Obteniendo resultados del job...")
        text_content = ""
        next_token = None
        all_lines = []  # Recopilar todas las líneas
        
        # Procesar todas las páginas de resultados
        while True:
            if next_token:
                result_response = textract_client.get_document_text_detection(
                    JobId=job_id,
                    NextToken=next_token
                )
            else:
                result_response = textract_client.get_document_text_detection(JobId=job_id)
            
            # Extraer líneas de los bloques
            blocks = result_response.get('Blocks', [])
            logger.info(f"Procesando {len(blocks)} bloques")
            
            # Recopilar solo bloques LINE (más simple)
            page_lines = {}
            for block in blocks:
                if block['BlockType'] == 'LINE':
                    page_num = block.get('Page', 1)
                    line_text = block.get('Text', '').strip()
                    
                    if line_text:  # Solo líneas no vacías
                        if page_num not in page_lines:
                            page_lines[page_num] = []
                        page_lines[page_num].append(line_text)
            
            # Agregar líneas por página a la colección total
            for page_num in sorted(page_lines.keys()):
                if page_num not in all_lines:
                    all_lines.append(f"--- PÁGINA {page_num} ---")
                
                for line in page_lines[page_num]:
                    all_lines.append(line)
            
            # Verificar si hay más páginas de resultados
            next_token = result_response.get('NextToken')
            if not next_token:
                break
                
            logger.info("Obteniendo siguiente página de resultados...")
        
        # PASO 3: CONSTRUIR TEXTO FINAL
        if all_lines:
            text_content = '\n'.join(all_lines)
        else:
            logger.warning("No se encontraron líneas de texto en la respuesta de Textract")
            text_content = "[SIN CONTENIDO DETECTADO POR TEXTRACT]"
        
        # Limpiar el texto extraído
        text_content = clean_text_for_json(text_content.strip())
        
        logger.info(f"Texto extraído con Textract exitosamente: {len(text_content)} caracteres")
        logger.info(f"Primeros 200 caracteres: {text_content[:200]}...")
        
        return text_content
        
    except Exception as e:
        logger.error(f"Error extrayendo texto con AWS Textract: {e}")
        return f"[ERROR EXTRAYENDO TEXTO CON TEXTRACT: {str(e)}]"

In [415]:
def _extract_text(resp_json: dict) -> str:
    """
    Bedrock Nova returns:
        {"output":{"message":{"content":[{"text":"..."}]}}}
    """
    try:
        text=resp_json["output"]["message"]["content"][0]["text"].strip()
        return clean_text_for_json(text)
    except (KeyError, IndexError, TypeError):
        raise RuntimeError("Unexpected response shape from Bedrock") from None

In [416]:
def get_first_pdf_page(pdf_bytes):
    """
    Extract the first page from a PDF (del proyecto original).
    """
    try:
        inputpdf = PdfReader(io.BytesIO(pdf_bytes))
        if len(inputpdf.pages) > 0:
            first_page = inputpdf.pages[0]
            writer = io.BytesIO()
            pdf_writer = PdfWriter()
            pdf_writer.add_page(first_page)
            pdf_writer.write(writer)
            return writer.getvalue()
        else:
            logger.warning("PDF has no pages")
            return pdf_bytes
    except Exception as e:
        logger.error(f"Error extracting first page: {str(e)}")
        return pdf_bytes

In [417]:
def detect_scanned_pdf(pdf_bytes: bytes, text_threshold: int = 20) -> bool:
    """
    Returns True if the PDF appears to be a scanned/image‐only PDF.
    We consider it scanned if no page yields more than text_threshold chars.
    """
    reader = PdfReader(io.BytesIO(pdf_bytes))
    for page in reader.pages:
        text = page.extract_text() or ""
        if len(text.strip()) > text_threshold:
            # Found enough text to call it "generated"
            return False
    # No substantial text on any page → likely scanned
    return True

In [418]:
def clean_dict_for_json(obj):
    """
    Recursively clean all string values in a dictionary/list structure
    to ensure JSON compatibility.
    """
    if isinstance(obj, dict):
        return {key: clean_dict_for_json(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [clean_dict_for_json(item) for item in obj]
    elif isinstance(obj, str):
        return clean_text_for_json(obj)
    else:
        return obj

In [419]:
@dataclass
class BedrockRequest:
    """Unified request structure for both APIs"""
    model_id: str
    messages: List[Dict[str, Any]]
    params: Dict[str, Any]
    system: Optional[List] = None
    toolConfig: Optional[Dict[str, Any]] = None

In [420]:
def is_nova_model(model_id: str) -> bool:
    """Check if the model is a Nova model"""
    return "nova" in model_id.lower()

In [421]:
def is_anthropic_model(model_id: str) -> bool:
    """Check if the model is an Anthropic model"""
    return "anthropic" in model_id.lower()

In [422]:
def set_model_params_unified_converse(max_tokens, top_p, temperature) -> Dict[str, Any]:
    """
    Set model parameters based on the API being used
    """
    return {"maxTokens": max_tokens,
        "topP": top_p,
        "temperature": temperature,
        }

In [423]:
def set_model_params_unified_anthropic(max_tokens, top_p, temperature) -> Dict[str, Any]:
    """
    Set model parameters based on the API being used
    """
    return {
        "max_tokens": max_tokens,
        "top_p": top_p,
        "temperature": temperature,
        "thinking": {
            "type": "enabled",
            "budget_tokens": 4096,
        },
    }


In [424]:
def create_converse_message(prompt: str, role: str, pdf_bytes: bytes = None, pdf_path: str = None) -> Dict[str, Any]:
    """Create message for Converse API"""
    content = [{"text": prompt}]
    
    if pdf_bytes is not None:
        name = sanitize_name(pdf_path)
        
        document_block = {
            "document": {
                "name": name,
                "format": "pdf",
                "source": {
                    "bytes": pdf_bytes
                }
            }
        }
        content.append(document_block)
    
    return {"role": role, "content": content}

In [425]:
def create_anthropic_message(prompt: str, role: str, pdf_bytes: bytes = None, pdf_path: str = None) -> Dict[str, Any]:
    """Create message for Anthropic InvokeModel API"""
    if pdf_bytes is not None:
        # Convert PDF to base64 for Anthropic
        pdf_base64 = base64.b64encode(pdf_bytes).decode('utf-8')
        
        return {
            "role": role,
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                    "cache_control":{"type":"ephemeral"}
                },
                {
                    "type": "document",
                    "source": {
                        "type": "base64",
                        "media_type": "application/pdf",
                        "data": pdf_base64
                    },
                    #"citations":{"enabled":True}
                    #"cache_control":{"type":"ephemeral"} #no ayudará mucho ya que lo que más se repite es el system prompt
                }
            ]
        }
    else:
        return {
            "role": role,
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }

In [426]:
def create_anthropic_message_with_text_fallback(prompt: str, role: str, pdf_text: str) -> Dict[str, Any]:
    """Create message for Anthropic InvokeModel API using extracted text instead of PDF"""
    return {
        "role": role,
        "content": [
            {
                    "type": "text",
                    "text": prompt,
                    "cache_control":{"type":"ephemeral"}
            },
            {
                    "type": "text",
                    "text": f"\n\n--- CONTENIDO DEL DOCUMENTO ---\n{pdf_text}"
            },
        ]
    }

In [427]:
def create_anthropic_message_with_textract_fallback(prompt: str, role: str, pdf_text: str) -> Dict[str, Any]:
    """Create message for Anthropic InvokeModel API using Textract extracted text as second fallback"""
    return {
        "role": role,
        "content": [
            {
                "type": "text",
                "text": prompt,
                "cache_control": {"type": "ephemeral"}
            },
            {
                "type": "text", 
                "text": f"\n\n--- CONTENIDO DEL DOCUMENTO ---\n{pdf_text}"
            },
        ]
    }

In [428]:
def is_input_too_long_error(error_message: str) -> bool:
    """Check if the error is specifically 'Input is too long for requested model'"""
    return ("Input is too long" in str(error_message) or 
            "too long" in str(error_message).lower() or
            "ValidationException" in str(error_message))

In [429]:
def create_message_unified(prompt: str, role: str, model_id: str, pdf_bytes: bytes = None, pdf_path: str = None) -> Dict[str, Any]:
    """
    Create message based on the target API
    """
    if is_anthropic_model(model_id):
        message = create_anthropic_message(prompt, role, pdf_bytes, pdf_path)
        
        # Log PDF size for debugging
        if pdf_bytes:
            logger.info(f"📄 PDF size for Anthropic model: {len(pdf_bytes)} bytes")
            
        return message
    else:
        return create_converse_message(prompt, role, pdf_bytes, pdf_path)

In [430]:
def call_converse_api(request: BedrockRequest, bedrock_client) -> Dict[str, Any]:
    """
    Call Bedrock Converse API (for Nova and other models)
    """
    payload = {
        "modelId": request.model_id,
        "messages": request.messages,
    }

    # Add inference config
    if request.params:
        payload["inferenceConfig"] = request.params

    # Add system prompt if provided
    if request.system:
        payload["system"] = request.system
        
    # Add tool config if provided
    if request.toolConfig:
        payload["toolConfig"] = request.toolConfig

    logger.info(f"Calling Converse API for model: {request.model_id}")
    return bedrock_client.converse(**payload)

In [431]:
def call_invoke_model_api(request: BedrockRequest, bedrock_client) -> Dict[str, Any]:
    """
    Call Bedrock InvokeModel API (for Anthropic models) with double fallback:
    1. Original PDF → 2. pdfplumber text → 3. Textract text
    """
    # Build Anthropic-specific payload
    payload = {
        "anthropic_version":"bedrock-2023-05-31",
        "messages": request.messages,
        **request.params  # max_tokens, top_p, temperature
    }
    
    logger.info(f"Calling InvokeModel API for model: {request.model_id}")
    
    try:
        # Primer intento con el payload original
        response = bedrock_client.invoke_model(
            modelId=request.model_id,
            body=json.dumps(payload),
            contentType="application/json"
        )
        
        # Parse InvokeModel response
        response_body = json.loads(response['body'].read())

        text_content = ""
        content_blocks = response_body.get('content', [])
            
        # Find the text content block (following the GitHub example pattern)
        for block in content_blocks:
            if block.get('type') == 'text':
                text_content = block.get('text', '')
                break
            
        if not text_content:
            raise ValueError(f"No text content found in response. Content blocks: {content_blocks}")
            
        logger.info(f"Successfully extracted text content ({len(text_content)} characters)")

        # Convert to Converse API format for consistency
        return {
            "output": {
                "message": {
                    "content": [
                        {
                            "text": text_content
                        }
                    ]
                }
            },
            "stopReason": response_body.get("stop_reason", "end_turn"),
            "usage": response_body.get("usage", {}),
            "ResponseMetadata": response.get('ResponseMetadata', {}),
            "raw_anthropic_response": response_body
        }
        
    except Exception as e:
        error_str = str(e)
        logger.warning(f"Error en primer intento: {error_str}")
        
        # Verificar si es el error específico de input demasiado largo
        if is_input_too_long_error(error_str):
            logger.info("🔄 Input too long detected - attempting FIRST fallback with pdfplumber")
            
            # Buscar si hay un documento PDF en los mensajes
            pdf_bytes = None
            pdf_found = False
            
            for message in request.messages:
                if message.get("role") == "user":
                    for content_item in message.get("content", []):
                        if content_item.get("type") == "document":
                            # Extraer el PDF bytes del documento
                            source = content_item.get("source", {})
                            if source.get("type") == "base64":
                                import base64
                                pdf_bytes = base64.b64decode(source.get("data", ""))
                                pdf_found = True
                                break
                    if pdf_found:
                        break
            
            if pdf_bytes:
                logger.info("📄 Extrayendo texto del PDF para PRIMER fallback (pdfplumber)...")
                
                # PRIMER FALLBACK: Extraer texto del PDF con pdfplumber
                extracted_text = extract_pdf_text_with_pdfplumber(pdf_bytes)
                
                # Crear nuevos mensajes sin el documento PDF, pero con el texto extraído
                new_messages = []
                for message in request.messages:
                    if message.get("role") == "user":
                        # Encontrar el prompt original (primer item de texto)
                        original_prompt = ""
                        for content_item in message.get("content", []):
                            if content_item.get("type") == "text":
                                original_prompt = content_item.get("text", "")
                                break
                        
                        # Crear mensaje nuevo solo con texto
                        new_message = create_anthropic_message_with_text_fallback(
                            original_prompt, 
                            message.get("role"), 
                            extracted_text
                        )
                        new_messages.append(new_message)
                    else:
                        # Mantener otros mensajes como están
                        new_messages.append(message)
                
                # Crear nuevo payload con el texto extraído por pdfplumber
                fallback_payload = {
                    "anthropic_version": "bedrock-2023-05-31",
                    "messages": new_messages,
                    **request.params
                }
                
                logger.info("🔄 Reintentando llamada con texto extraído por pdfplumber...")
                
                try:
                    # PRIMER FALLBACK: Segundo intento con texto extraído por pdfplumber
                    fallback_response = bedrock_client.invoke_model(
                        modelId=request.model_id,
                        body=json.dumps(fallback_payload),
                        contentType="application/json"
                    )
                    
                    # Parse fallback response
                    fallback_response_body = json.loads(fallback_response['body'].read())
                    
                    fallback_text_content = ""
                    fallback_content_blocks = fallback_response_body.get('content', [])
                        
                    for block in fallback_content_blocks:
                        if block.get('type') == 'text':
                            fallback_text_content = block.get('text', '')
                            break
                        
                    if not fallback_text_content:
                        raise ValueError(f"No text content found in pdfplumber fallback response. Content blocks: {fallback_content_blocks}")
                    
                    logger.info(f"✅ PRIMER fallback (pdfplumber) successful - extracted text content ({len(fallback_text_content)} characters)")
                    
                    # Return successful first fallback response
                    return {
                        "output": {
                            "message": {
                                "content": [
                                    {
                                        "text": fallback_text_content
                                    }
                                ]
                            }
                        },
                        "stopReason": fallback_response_body.get("stop_reason", "end_turn"),
                        "usage": fallback_response_body.get("usage", {}),
                        "ResponseMetadata": fallback_response.get('ResponseMetadata', {}),
                        "raw_anthropic_response": fallback_response_body,
                        "fallback_used": True,
                        "fallback_reason": "input_too_long",
                        "fallback_method": "pdfplumber",
                        "extracted_text_length": len(extracted_text)
                    }
                    
                except Exception as first_fallback_error:
                    logger.error(f"❌ PRIMER fallback (pdfplumber) también falló: {first_fallback_error}")
                    
                    # ===========================================
                    # SEGUNDO FALLBACK: AWS TEXTRACT
                    # ===========================================
                    logger.info("🔄 Attempting SECOND fallback with AWS Textract...")
                    
                    try:
                        # SEGUNDO FALLBACK: Extraer texto con AWS Textract
                        textract_text = extract_pdf_text_with_textract(
                            pdf_bytes, 
                            os.environ.get("REGION"), 
                            os.environ.get("AWS_PROFILE")
                        )
                        
                        # Crear nuevos mensajes con texto de Textract
                        textract_messages = []
                        for message in request.messages:
                            if message.get("role") == "user":
                                # Encontrar el prompt original (primer item de texto)
                                original_prompt = ""
                                for content_item in message.get("content", []):
                                    if content_item.get("type") == "text":
                                        original_prompt = content_item.get("text", "")
                                        break
                                
                                # Crear mensaje nuevo con texto de Textract
                                textract_message = create_anthropic_message_with_textract_fallback(
                                    original_prompt, 
                                    message.get("role"), 
                                    textract_text
                                )
                                textract_messages.append(textract_message)
                            else:
                                # Mantener otros mensajes como están
                                textract_messages.append(message)
                        
                        # Crear payload con texto de Textract
                        textract_payload = {
                            "anthropic_version": "bedrock-2023-05-31",
                            "messages": textract_messages,
                            **request.params
                        }
                        
                        logger.info("🔄 Reintentando llamada con texto extraído por Textract...")
                        
                        # SEGUNDO FALLBACK: Tercer intento con texto extraído por Textract
                        textract_response = bedrock_client.invoke_model(
                            modelId=request.model_id,
                            body=json.dumps(textract_payload),
                            contentType="application/json"
                        )
                        
                        # Parse Textract fallback response
                        textract_response_body = json.loads(textract_response['body'].read())
                        
                        textract_text_content = ""
                        textract_content_blocks = textract_response_body.get('content', [])
                            
                        for block in textract_content_blocks:
                            if block.get('type') == 'text':
                                textract_text_content = block.get('text', '')
                                break
                            
                        if not textract_text_content:
                            raise ValueError(f"No text content found in Textract fallback response. Content blocks: {textract_content_blocks}")
                        
                        logger.info(f"✅ SEGUNDO fallback (Textract) successful - extracted text content ({len(textract_text_content)} characters)")
                        
                        # Return successful second fallback response
                        return {
                            "output": {
                                "message": {
                                    "content": [
                                        {
                                            "text": textract_text_content
                                        }
                                    ]
                                }
                            },
                            "stopReason": textract_response_body.get("stop_reason", "end_turn"),
                            "usage": textract_response_body.get("usage", {}),
                            "ResponseMetadata": textract_response.get('ResponseMetadata', {}),
                            "raw_anthropic_response": textract_response_body,
                            "fallback_used": True,
                            "fallback_reason": "input_too_long_and_pdfplumber_failed",
                            "fallback_method": "textract",
                            "extracted_text_length": len(textract_text),
                            "first_fallback_error": str(first_fallback_error)
                        }
                        
                    except Exception as textract_error:
                        logger.error(f"❌ SEGUNDO fallback (Textract) también falló: {textract_error}")
                        raise RuntimeError(
                            f"All attempts failed. "
                            f"Original: {error_str}, "
                            f"First fallback (pdfplumber): {str(first_fallback_error)}, "
                            f"Second fallback (Textract): {str(textract_error)}"
                        )
            else:
                logger.error("❌ No se encontró documento PDF para extraer texto en fallbacks")
                raise RuntimeError(f"Input too long error but no PDF document found for text extraction: {error_str}")
        else:
            # Re-raise the original error if it's not the input too long error
            raise

In [432]:
def call_bedrock_unified(request: BedrockRequest, bedrock_client) -> Dict[str, Any]:
    """
    Unified function to call appropriate Bedrock API based on model type
    """
    if is_anthropic_model(request.model_id):
        response = call_invoke_model_api(request, bedrock_client)
    else:
        response = call_converse_api(request, bedrock_client)
    
    # Add metadata for tracking
    if isinstance(response, dict):
        response['model_id'] = request.model_id
        response['api_used'] = 'invoke_model' if is_anthropic_model(request.model_id) else 'converse'
        response['model_params'] = request.params
    
    return response

In [433]:
def parse_classification_response(response: Dict[str, Any], folder_path: str = None) -> Dict[str, Any]:
    """Parsear respuesta de clasificación (CORREGIDO - igual que proyecto principal)"""
    raw_text = _extract_text(response)
    raw_text = _strip_fences(raw_text)
    raw_obj = None  # ← Inicializar la variable

    try:
        raw_obj = json.loads(raw_text)
        print(f"Raw JSON: {raw_obj}")  # Debugging output
    except json.JSONDecodeError as e:
        # last-chance: grab first '{' … last '}'
        m1, m2 = raw_text.find("{"), raw_text.rfind("}")
        if m1 != -1 and m2 != -1:
            try:
                raw_obj = json.loads(raw_text[m1:m2+1])
            except Exception as inner_e:
                raise RuntimeError(f"Failed to parse JSON even after extraction: {inner_e}. Original error: {e}") from None
        else:
            raise RuntimeError(f"Assistant did not return JSON: {e}") from None

    # Verificar que raw_obj se definió correctamente
    if raw_obj is None:
        raise RuntimeError("Failed to parse JSON response") from None

    return _normalise(raw_obj, file_path=folder_path)

In [434]:
def parse_extraction_response(response: Dict[str, Any]) -> Dict[str, Any]:
    """Parsear respuesta de extracción (del proyecto real)"""
    try:
        text = response["output"]["message"]["content"][0]["text"]
        
        # Buscar JSON en ```json ... ```
        import re
        json_match = re.search(r'```json\s*(\{.*?\})\s*```', text, re.DOTALL)
        if json_match:
            return json.loads(json_match.group(1))
        else:
            # Intentar parsear todo como JSON
            return json.loads(text.strip())
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        raise ValueError(f"Error parseando respuesta de extracción: {e}")

In [435]:
@dataclass
class TestDocument:
    """Documento de prueba"""
    name: str
    category: str
    pdf_path: Path
    pdf_bytes: bytes
    expected_category: str
    document_number: str
    subfolder_path: Path

In [436]:
def parse_classification_response_fallback(response: Dict[str, Any], folder_path: str = None, test_doc: TestDocument = None) -> Dict[str, Any]:
    """
    Método de parsing alternativo que identifica y corrige caracteres corruptos específicos
    que causan errores de JSON parsing, sin adivinar contenido
    """
    try:
        # Extraer el texto crudo de la respuesta
        raw_text = _extract_text(response)
        logger.info(f"🔄 Intentando parsing alternativo para corregir caracteres JSON corruptos")
        
        # Remover las vallas de código
        cleaned_text = _strip_fences(raw_text)
        
        # PASO 1: Identificar y corregir escapes malformados específicos que causan errores JSON
        
        # Corregir escapes de comillas malformados: \\\" -> \"
        cleaned_text = re.sub(r'\\{2,}"', r'"', cleaned_text)  # \\\" o \\\\" -> "
        
        # Corregir secuencias de escapes múltiples: \\\\n -> \n, \\\\t -> \t
        cleaned_text = re.sub(r'\\{2,}n', r'\\n', cleaned_text)  # \\\\n -> \n
        cleaned_text = re.sub(r'\\{2,}t', r'\\t', cleaned_text)  # \\\\t -> \t
        
        # Corregir escapes de backslash malformados: \\\\ -> \\
        cleaned_text = re.sub(r'\\{3,}', r'\\\\', cleaned_text)  # \\\\\ -> \\
        
        # PASO 2: Intentar parsear el JSON corregido
        try:
            raw_obj = json.loads(cleaned_text)
            logger.info(f"✅ JSON corregido parseado exitosamente")
        except json.JSONDecodeError as json_error:
            logger.info(f"⚠️  JSON aún no válido después de correcciones básicas: {json_error}")
            
            # PASO 3: Si aún falla, aplicar correcciones más agresivas solo en el campo "text"
            # Extraer category (que generalmente no está corrupto)
            category_match = re.search(r'"category"\s*:\s*"([^"]+)"', cleaned_text)
            category = category_match.group(1) if category_match else "UNKNOWN"
            
            # Para el campo text, extraer solo el contenido entre las últimas comillas válidas
            # Buscar el patrón: "text": "..." 
            text_match = re.search(r'"text"\s*:\s*"(.*)"(?:\s*}?\s*$)', cleaned_text, re.DOTALL)
            
            if text_match:
                text_content = text_match.group(1)
                
                # Limpiar caracteres de escape problemáticos SOLO en el contenido del text
                text_content = text_content.replace('\\"', '"')  # \" -> "
                text_content = re.sub(r'\\+$', '', text_content)  # Remover backslashes al final
                
                # Reconstruir JSON válido
                reconstructed_json = f'{{"category": "{category}", "text": "{text_content}"}}'
                
                try:
                    raw_obj = json.loads(reconstructed_json)
                    logger.info(f"✅ JSON reconstruido parseado exitosamente")
                except json.JSONDecodeError:
                    # Si la reconstrucción falla, usar valores básicos
                    raw_obj = {
                        "category": category,
                        "text": text_content[:500] + "..." if len(text_content) > 500 else text_content
                    }
                    logger.info(f"⚠️  Usando JSON simplificado con contenido truncado")
            else:
                # Si no podemos extraer text, usar valores por defecto
                raw_obj = {
                    "category": category,
                    "text": "[CONTENIDO NO PARSEABLE - CARACTERES CORRUPTOS]"
                }
                logger.warning(f"⚠️  No se pudo extraer campo text, usando placeholder")
        
        # PASO 4: Usar _normalise para agregar document_number y path
        result = _normalise(raw_obj, file_path=folder_path)
        
        logger.info(f"✅ Parsing alternativo exitoso")
        logger.info(f"    Categoría: {result.get('category', 'UNKNOWN')}")
        logger.info(f"    Texto: {len(result.get('text', ''))} caracteres")
        
        return result
        
    except Exception as e:
        logger.error(f"❌ Error en parsing alternativo: {e}")
        # Si todo falla, devolver estructura mínima para _normalise
        raw_obj = {
            "category": "UNKNOWN", 
            "text": f"ERROR_FALLBACK_PARSING: {str(e)}"
        }
        return _normalise(raw_obj, file_path=folder_path)

In [437]:
def parse_classification_response_robust(response: Dict[str, Any], folder_path: str = None, test_doc: TestDocument=None) -> Dict[str, Any]:
    """Parse classification response with robust error handling"""
    try:
        return parse_classification_response(response, folder_path)
    except (json.JSONDecodeError, RuntimeError) as e:
        logger.error(f"❌ Error parsing classification response: {e}")
        
        # Save raw response to file for debugging
        if test_doc:
            raw_response_dir = PROJECT_STRUCTURE_ROOT / "results" / "raw_responses" / "classification" / test_doc.document_number
            raw_response_dir.mkdir(parents=True, exist_ok=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            raw_file = raw_response_dir / f"raw_classification_response_{timestamp}.json"
            
            with open(raw_file, 'w', encoding='utf-8') as f:
                json.dump(response, f, indent=2, ensure_ascii=False, default=str)
            
            logger.info(f"💾 Raw classification response saved: {raw_file}")
        
        # NUEVO: Intentar parsing alternativo antes de devolver fallback genérico
        logger.info(f"🔄 Intentando método de parsing alternativo...")
        try:
            fallback_result = parse_classification_response_fallback(response, folder_path, test_doc)
            if fallback_result.get("category") != "UNKNOWN":
                logger.info(f"✅ Parsing alternativo exitoso - Categoría detectada: {fallback_result['category']}")
                return fallback_result
            else:
                logger.warning(f"⚠️  Parsing alternativo no pudo determinar categoría específica")
        except Exception as fallback_error:
            logger.error(f"❌ Error en parsing alternativo: {fallback_error}")
        
        # Si el parsing alternativo también falla, devolver fallback genérico
        return {
            "document_number": test_doc.document_number if test_doc else "UNKNOWN",
            "document_type": "company",
            "category": "UNKNOWN",
            "text": "ERROR_PARSING_RESPONSE",
            "path": folder_path or "UNKNOWN",
            "parse_error": str(e),
            "has_raw_response": True
        }

In [438]:
def parse_extraction_response_robust(response: Dict[str, Any], test_doc: TestDocument = None, category: str = None) -> Dict[str, Any]:
    """Parse extraction response with robust error handling"""
    try:
        return parse_extraction_response(response)
    except (json.JSONDecodeError, ValueError) as e:
        logger.error(f"❌ Error parsing extraction response: {e}")
        
        # Save raw response to file for debugging
        if test_doc:
            raw_response_dir = PROJECT_STRUCTURE_ROOT / "results" / "raw_responses" / "extraction" / test_doc.document_number
            raw_response_dir.mkdir(parents=True, exist_ok=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            raw_file = raw_response_dir / f"raw_extraction_response_{category}_{timestamp}.json"
            
            with open(raw_file, 'w', encoding='utf-8') as f:
                json.dump(response, f, indent=2, ensure_ascii=False, default=str)
            
            logger.info(f"💾 Raw extraction response saved: {raw_file}")
        
        # Return a fallback extraction result
        return {
            "path": f"{category}/{test_doc.document_number}/{test_doc.name}" if test_doc and category else "UNKNOWN",
            "result": {"error": "PARSE_ERROR"},
            "confidenceScores": {"error": 0},
            "document_type": "company",
            "document_number": test_doc.document_number if test_doc else "UNKNOWN",
            "category": category or "UNKNOWN",
            "parse_error": str(e),
            "has_raw_response": True
        }

In [439]:
class DocumentLoader:
    """Cargador de documentos PDF reales"""
    
    def __init__(self, test_docs_root: Path):
        self.test_docs_root = test_docs_root
        
    def load_documents_from_folder(self, category: str) -> List[TestDocument]:
        """Cargar documentos de una carpeta específica"""
        category_path = self.test_docs_root / category
        if not category_path.exists():
            logger.warning(f"Carpeta no existe: {category_path}")
            return []
        documents = []
        logger.info(f"🔍 Buscando subcarpetas numeradas en: {category_path}")
        
        # Buscar subcarpetas numeradas
        subfolders_found = []
        for subfolder in category_path.iterdir():
            if not subfolder.is_dir():
                logger.debug(f"Saltando archivo en {category}: {subfolder.name}")
                continue
            
            # Verificar que el nombre de la subcarpeta contenga números
            if not re.search(r'\d+', subfolder.name):
                logger.debug(f"Saltando subcarpeta no numérica en {category}: {subfolder.name}")
                continue
            
            subfolders_found.append(subfolder)
        
        if not subfolders_found:
            logger.warning(f"❌ No se encontraron subcarpetas numeradas en {category_path}")
            logger.info(f"   Estructura esperada: {category_path}/[NUMERO]/archivo.pdf")
            return []
        
        logger.info(f"✅ Encontradas {len(subfolders_found)} subcarpetas numeradas en {category}")
        
        # Procesar cada subcarpeta
        for subfolder in sorted(subfolders_found, key=lambda x: x.name):
            logger.info(f"📂 Procesando subcarpeta: {category}/{subfolder.name}")
            
            # Buscar archivos PDF dentro de la subcarpeta
            pdf_files = list(subfolder.glob("*.pdf"))
            
            if len(pdf_files) == 0:
                logger.warning(f"⚠️  No se encontró PDF en {category}/{subfolder.name}")
                self._show_folder_contents(subfolder)
                continue
            logger.info(f"✅ Encontrados {len(pdf_files)} PDFs en {category}/{subfolder.name}")
            for i, pdf_file in enumerate(pdf_files):
                logger.info(f"    PDF {i+1}/{len(pdf_files)}: {pdf_file.name}")
            
            # Procesar cada PDF individualmente
            for pdf_index, pdf_file in enumerate(pdf_files):
                try:
                    # Leer bytes del PDF
                    with open(pdf_file, 'rb') as f:
                        pdf_bytes = f.read()
                    
                    base_doc_number = subfolder.name
                    if len(pdf_files) > 1:
                        # Si hay múltiples PDFs, agregar sufijo para diferenciación
                        pdf_stem = pdf_file.stem
                        doc_number = f"{base_doc_number}_{pdf_stem}"
                    else:
                        # Si solo hay un PDF, usar el número de la subcarpeta
                        doc_number = base_doc_number

                    doc = TestDocument(
                        name=pdf_file.name,
                        category=category,
                        pdf_path=pdf_file,
                        pdf_bytes=pdf_bytes,
                        expected_category=category,
                        document_number=doc_number,
                        subfolder_path=subfolder
                    )
                    documents.append(doc)
                    logger.info(f"📄 Cargado: {pdf_file.name} ({len(pdf_bytes)} bytes)")
                    
                except Exception as e:
                    logger.error(f"Error cargando {pdf_file}: {e}")
            
        return documents

    def _show_folder_contents(self, folder_path: Path, max_files: int = 5):
        """Mostrar contenido de una carpeta para debugging"""
        try:
            files = list(folder_path.iterdir())
            if files:
                logger.info(f"    Contenido de {folder_path.name}:")
                for i, file in enumerate(files[:max_files]):
                    file_type = "📁" if file.is_dir() else "📄"
                    logger.info(f"      {file_type} {file.name}")
                if len(files) > max_files:
                    logger.info(f"      ... y {len(files) - max_files} más")
            else:
                logger.info(f"    Carpeta vacía: {folder_path.name}")
        except Exception as e:
            logger.warning(f"    Error leyendo contenido: {e}")
    
    def load_all_documents_ordered(self, categories: List[str] = None) -> Dict[str, List[TestDocument]]:
        """Cargar documentos en el orden especificado para optimizar caching"""
        if categories is None:
            categories = CATEGORIES_ORDER
            
        all_docs = {}
        
        for category in categories:
            docs = self.load_documents_from_folder(category)
            if docs:
                all_docs[category] = docs
                logger.info(f"✅ {category}: {len(docs)} documentos cargados")
                
                subfolders = {}
                for doc in docs:
                    subfolder = doc.subfolder_path.name
                    if subfolder not in subfolders:
                        subfolders[subfolder] = []
                    subfolders[subfolder].append(doc.name)
                
                for subfolder, files in subfolders.items():
                    logger.info(f"   📂 {subfolder}: {len(files)} archivo(s) - {', '.join(files)}")
            else:
                logger.info(f"❌ {category}: Sin documentos")
        
        if not all_docs:
            print("\n❌ NO SE ENCONTRARON DOCUMENTOS")
            print("📁 Estructura esperada:")
            print("   test_documents/")
            print("   ├── ACC/")
            print("   │   ├── 800035887/")
            print("   │   │   ├── certificado.pdf  ← Este se procesará")
            print("   │   └── 900123456/")
            print("   │       └── documento.pdf  ← Este se procesará")
            print("   ├── CECRL/")
            print("   │   ├── 001/")
            print("   │   │   └── cedula.pdf  ← Este se procesará")
            print("   └── ...")
            
        return all_docs

    def show_structure_summary(self) -> None:
        """Mostrar resumen de la estructura detectada"""
        print("\n📊 RESUMEN DE ESTRUCTURA:")
        print("="*40)
        
        total_subfolders = 0
        total_pdfs = 0
        
        for category in CATEGORIES_ORDER:
            category_path = self.test_docs_root / category
            if not category_path.exists():
                print(f"❌ {category}: Carpeta no existe")
                continue
            
            # Contar subcarpetas numeradas
            subfolders = [f for f in category_path.iterdir() 
                         if f.is_dir() and re.search(r'\d+', f.name)]
            
            # Contar PDFs en subcarpetas
            pdfs_found = 0
            subfolders_with_pdfs = 0
            for subfolder in subfolders:
                pdfs_in_folder = len(list(subfolder.glob("*.pdf")))
                if pdfs_in_folder > 0:
                    subfolders_with_pdfs += 1
                    pdfs_found += pdfs_in_folder  # Sumar TODOS los PDFs
            
            total_subfolders += len(subfolders)
            total_pdfs += pdfs_found
            
            status = "✅" if pdfs_found > 0 else "❌"
            print(f"{status} {category}: {len(subfolders)} subcarpetas, {pdfs_found} con PDFs")
        
        print(f"\n📈 TOTAL: {total_subfolders} subcarpetas, {total_pdfs} documentos procesables")



In [440]:
class PromptSaver:
    """Saver for complete prompts for analysis"""
    
    def __init__(self, project_root: Path):
        self.prompts_dir = project_root / "results"/"prompts_completos"
        self.prompts_dir.mkdir(exist_ok=True)
        
        # Create subdirectories
        (self.prompts_dir / "classification").mkdir(exist_ok=True)
        (self.prompts_dir / "extraction").mkdir(exist_ok=True)
    
    def save_classification_prompts(self, test_doc: TestDocument, system_prompt: str, user_prompt: str):
        """Save classification prompts"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        doc_folder = self.prompts_dir / "classification" / test_doc.document_number
        doc_folder.mkdir(exist_ok=True)

        base_name = f"{test_doc.category}_{test_doc.document_number}_{timestamp}"
        
        # Save system prompt
        system_file = doc_folder / f"{base_name}_system.txt"

        with open(system_file, 'w', encoding='utf-8') as f:
            f.write(system_prompt)
        
        # Save user prompt
        user_file = doc_folder / f"{base_name}_user.txt"
        with open(user_file, 'w', encoding='utf-8') as f:
            f.write(user_prompt)
        
        logger.info(f"💾 Classification prompts saved: {base_name}")
    
    def save_extraction_prompts(self, test_doc: TestDocument, category: str, system_prompt: str, user_prompt: str):
        """Save extraction prompts"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        doc_folder = self.prompts_dir / "extraction" / test_doc.document_number
        doc_folder.mkdir(exist_ok=True)

        base_name = f"{category}_{timestamp}"
        
        # Save system prompt
        system_file = doc_folder / f"{base_name}_system.txt"
        with open(system_file, 'w', encoding='utf-8') as f:
            f.write(system_prompt)
        
        # Save user prompt
        user_file = doc_folder / f"{base_name}_user.txt"
        with open(user_file, 'w', encoding='utf-8') as f:
            f.write(user_prompt)
        
        logger.info(f"💾 Extraction prompts saved: {base_name}")

In [441]:
class DocumentTesterRefactored:
    """Updated DocumentTester with refactored API handling and fixed template issues"""
    def __init__(self, project_root: Path, use_real_bedrock: bool = True, model_id: str = None):
        self.project_root = project_root
        self.use_real_bedrock = use_real_bedrock
        self.prompt_saver = PromptSaver(project_root)
        
        # Configure model_id - use parameter or environment variable
        self.model_id = model_id or os.environ.get("BEDROCK_MODEL")
        if not self.model_id:
            raise ValueError("Must specify model_id or configure BEDROCK_MODEL in environment")
            
        logger.info(f"🤖 Model configured: {self.model_id}")
        
        if use_real_bedrock:
            try:
                self.bedrock_adm, self.bedrock_client = create_bedrock_client(os.environ["REGION"], os.environ["AWS_PROFILE"])
                logger.info("✅ Real Bedrock client connected")
            except Exception as e:
                logger.error(f"❌ Error connecting Bedrock: {e}")
                raise
        else:
            logger.info("🧪 Testing mode without Bedrock")
            self.bedrock_client = None
        self._classification_prompts = None
        self._extraction_prompts_cache = {}

    def get_classification_prompts(self) -> Tuple[str, str]:
        """Get classification prompts (cached)"""
        if self._classification_prompts is None:
            system_path = self.project_root / "classification/src/instructions/system.txt"
            user_path = self.project_root / "classification/src/instructions/user.txt"
            
            system_prompt = system_path.read_text(encoding='utf-8')
            user_prompt = user_path.read_text(encoding='utf-8')
            
            self._classification_prompts = (system_prompt, user_prompt)
            logger.info("📋 Classification prompts cached")
            
        return self._classification_prompts
    
    def get_extraction_prompts(self, category: str) -> Tuple[str, str]:
        """Get extraction prompts for a category (cached)"""
        if category not in self._extraction_prompts_cache:
            system_path = self.project_root / "extraction-scoring/src/instructions" / category / "system.txt"
            user_path = self.project_root / "extraction-scoring/src/instructions" / category / "user.txt"
            
            system_prompt = system_path.read_text(encoding='utf-8')
            user_prompt = user_path.read_text(encoding='utf-8')
            
            self._extraction_prompts_cache[category] = (system_prompt, user_prompt)
            logger.info(f"📋 Extraction prompts for {category} cached")
        
        return self._extraction_prompts_cache[category]

    def classify_document(self, test_doc: TestDocument) -> Tuple[bool, Dict[str, Any]]:
        """Clasificar un solo documento"""
        logger.info(f"🔍 Clasificando: {test_doc.category}/{test_doc.document_number}/{test_doc.name}")
        
        try:
            system_prompt, user_prompt = self.get_classification_prompts()
            
            # Solo guardar prompts la primera vez por documento
            self.prompt_saver.save_classification_prompts(test_doc, system_prompt, user_prompt)
            
            first_page_bytes = get_first_pdf_page(test_doc.pdf_bytes)
            is_scanned = detect_scanned_pdf(test_doc.pdf_bytes)
            
            # Create message using unified function
            messages = [create_message_unified(user_prompt, "user", self.model_id, first_page_bytes, test_doc.name)]
            
            if is_anthropic_model(self.model_id):
                params = set_model_params_unified_anthropic(8000, 1, 1)
                request = BedrockRequest(
                    model_id=self.model_id,
                    messages=messages,
                    params=params,
                    system=[{"text": system_prompt}]
                )
            else:
                params = set_model_params_unified_converse(8000, 1, 0)
                request = BedrockRequest(
                    model_id=self.model_id,
                    messages=messages,
                    params=params,
                    system=[
                        {"text": system_prompt},
                        {"cachePoint": {"type": "default"}}
                    ]
                )
            
            response = call_bedrock_unified(request, self.bedrock_client)
            folder_path = f"{test_doc.category}/{test_doc.document_number}/{test_doc.name}"
            classification_data = parse_classification_response_robust(response, folder_path, test_doc)
            
            success = classification_data.get("category") == test_doc.expected_category
            
            return success, {
                "classification_data": classification_data,
                "raw_response": response,
                "expected_category": test_doc.expected_category,
                "model_used": self.model_id,
                "api_used": response.get('api_used', 'unknown'),
                "pdf_info": {
                    "original_size": len(test_doc.pdf_bytes),
                    "first_page_size": len(first_page_bytes),
                    "is_scanned": is_scanned
                }
            }   
        except Exception as e:
            logger.error(f"❌ Error en clasificación: {e}")
            return False, {"error": str(e)}
    
    def extract_from_document(self, test_doc: TestDocument, classification_data: Dict[str, Any]) -> Tuple[bool, Dict[str, Any]]:
        """Extraer información de un documento ya clasificado"""
        category = classification_data.get("category", test_doc.category)
        logger.info(f"🔧 Extrayendo: {test_doc.category}/{test_doc.document_number}/{test_doc.name} → {category}")
        
        try:
            if category in ["BLANK", "LINK_ONLY"]:
                return True, {"skipped": True, "reason": "Non-extractable category"}
            
            system_prompt, user_prompt = self.get_extraction_prompts(category)
            
            # Solo guardar prompts la primera vez por documento/categoría
            self.prompt_saver.save_extraction_prompts(test_doc, category, system_prompt, user_prompt)
            
            messages = [create_message_unified(user_prompt, "user", self.model_id, test_doc.pdf_bytes, test_doc.name)]
            
            if is_anthropic_model(self.model_id):
                params = set_model_params_unified_anthropic(9000, 1, 1)
            else:
                params = set_model_params_unified_converse(2500, 1, 0)
            
            request = BedrockRequest(
                model_id=self.model_id,
                messages=messages,
                params=params,
                system=[{"text": system_prompt}]
            )
            
            response = call_bedrock_unified(request, self.bedrock_client)
            extraction_data = parse_extraction_response_robust(response, test_doc, category)
            
            return True, {
                "extraction_data": extraction_data,
                "raw_response": response,
                "model_used": self.model_id,
                "api_used": response.get('api_used', 'unknown')
            }
            
        except Exception as e:
            logger.error(f"❌ Error en extracción: {e}")
            return False, {"error": str(e)}
    
    def test_complete_flow(self, test_doc: 'TestDocument') -> Dict[str, Any]:
        """Test complete flow: classification + extraction"""
        logger.info(f"🚀 Complete flow: {test_doc.name}")
        
        result = {
            "document_name": test_doc.name,
            "category": test_doc.category,
            "classification_success": False,
            "extraction_success": False,
            "overall_success": False,
            "timestamp": datetime.now().isoformat()
        }
        
        # Step 1: Classification
        class_success, class_result = self.test_classification(test_doc)
        result["classification_success"] = class_success
        result["classification_result"] = class_result
        
        if not class_success:
            result["error"] = "Classification failed"
            return result
        
        # Step 2: Extraction
        extract_success, extract_result = self.test_extraction(test_doc, class_result)
        result["extraction_success"] = extract_success
        result["extraction_result"] = extract_result
        
        if not extract_success:
            result["error"] = "Extraction failed"
            return result
        
        result["overall_success"] = True
        return result

In [442]:
class ReportGeneratorOptimized:
    """Generador de reportes optimizado para el nuevo flujo"""
    
    def __init__(self, results_path: Path):
        self.results_path = results_path
        self.classification_path = results_path / "classification"
        self.extraction_path = results_path / "extraction"
        
        # Asegurar que existen las carpetas
        self.results_path.mkdir(exist_ok=True)
        self.classification_path.mkdir(exist_ok=True)
        self.extraction_path.mkdir(exist_ok=True)
        
    def save_classification_result(self, classification_result: Dict[str, Any], test_doc: TestDocument):
        """Guardar resultado de clasificación"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Limpiar datos antes de guardar
        cleaned_result = clean_dict_for_json(classification_result)
        
        base_doc_number = test_doc.subfolder_path.name
        class_doc_folder = self.classification_path / base_doc_number
        class_doc_folder.mkdir(exist_ok=True)
        
        safe_doc_name = re.sub(r'[^\w\-_.]', '_', test_doc.name)
        class_filename = f"classification_{test_doc.category}_{safe_doc_name}_{timestamp}.json"
        class_filepath = class_doc_folder / class_filename
        
        with open(class_filepath, 'w', encoding='utf-8') as f:
            json.dump(cleaned_result, f, indent=2, ensure_ascii=False, default=str)
        
        logger.info(f"💾 Clasificación guardada: {base_doc_number}/{class_filename}")
    
    def save_extraction_result(self, extraction_result: Dict[str, Any], test_doc: TestDocument, category: str):
        """Guardar resultado de extracción"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Limpiar datos antes de guardar
        cleaned_result = clean_dict_for_json(extraction_result)
        
        base_doc_number = test_doc.subfolder_path.name
        extract_doc_folder = self.extraction_path / base_doc_number
        extract_doc_folder.mkdir(exist_ok=True)
        
        safe_doc_name = re.sub(r'[^\w\-_.]', '_', test_doc.name)
        extract_filename = f"extraction_{category}_{safe_doc_name}_{timestamp}.json"
        extract_filepath = extract_doc_folder / extract_filename
        
        with open(extract_filepath, 'w', encoding='utf-8') as f:
            json.dump(cleaned_result, f, indent=2, ensure_ascii=False, default=str)
        
        logger.info(f"💾 Extracción guardada: {base_doc_number}/{extract_filename}")
    
    def save_final_summary(self, all_results: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Generar y guardar resumen final"""
        total_tests = len(all_results)
        classification_successes = sum(1 for r in all_results if r.get("classification_success", False))
        extraction_successes = sum(1 for r in all_results if r.get("extraction_success", False))
        overall_successes = sum(1 for r in all_results if r.get("overall_success", False))
        
        # Agrupar por categoría
        by_category = {}
        for result in all_results:
            category = result.get("category", "UNKNOWN")
            if category not in by_category:
                by_category[category] = []
            by_category[category].append(result)
        
        category_stats = {}
        for category, results in by_category.items():
            category_stats[category] = {
                "total": len(results),
                "classification_success": sum(1 for r in results if r.get("classification_success", False)),
                "extraction_success": sum(1 for r in results if r.get("extraction_success", False)),
                "overall_success": sum(1 for r in results if r.get("overall_success", False))
            }
        
        summary = {
            "summary": {
                "total_tests": total_tests,
                "total_pdfs_processed": total_tests,
                "classification_success_rate": classification_successes / total_tests if total_tests > 0 else 0,
                "extraction_success_rate": extraction_successes / total_tests if total_tests > 0 else 0,
                "overall_success_rate": overall_successes / total_tests if total_tests > 0 else 0,
                "timestamp": datetime.now().isoformat(),
                "processing_mode": "optimized_for_prompt_caching"
            },
            "by_category": category_stats,
            "detailed_results": all_results
        }
        
        # Limpiar el resumen antes de guardarlo
        cleaned_summary = clean_dict_for_json(summary)
        
        # Guardar resumen
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        summary_filename = f"testing_summary_optimized_{timestamp}.json"
        summary_path = self.results_path / summary_filename
        
        with open(summary_path, 'w', encoding='utf-8') as f:
            json.dump(cleaned_summary, f, indent=2, ensure_ascii=False, default=str)
        
        logger.info(f"📊 Resumen final guardado: {summary_filename}")
        return cleaned_summary
    
    def print_summary(self, summary: Dict[str, Any]):
        """Imprimir resumen en consola"""
        print("\n" + "="*70)
        print("📊 RESUMEN DE TESTING - PAR SERVICIOS (OPTIMIZADO PARA CACHING)")
        print("="*70)
        
        stats = summary["summary"]
        print(f"Total de pruebas: {stats['total_tests']}")
        print(f"Tasa de éxito en clasificación: {stats['classification_success_rate']:.1%}")
        print(f"Tasa de éxito en extracción: {stats['extraction_success_rate']:.1%}")
        print(f"Tasa de éxito general: {stats['overall_success_rate']:.1%}")
        print(f"Modo de procesamiento: {stats.get('processing_mode', 'unknown')}")
        
        print(f"\n📋 Resultados por categoría:")
        for category in CATEGORIES_ORDER:
            if category in summary["by_category"]:
                cat_stats = summary["by_category"][category]
                success_rate = cat_stats["overall_success"] / cat_stats["total"] if cat_stats["total"] > 0 else 0
                print(f"  {category}: {cat_stats['overall_success']}/{cat_stats['total']} ({success_rate:.1%})")
        
        print(f"\n🔍 Resultados detallados:")
        for result in summary["detailed_results"]:
            status = "✅ PASS" if result.get("overall_success", False) else "❌ FAIL"
            print(f"  {status} - {result['document_name']} ({result['category']})")
            
            if not result.get("overall_success", False) and "error" in result:
                print(f"    Error: {result['error']}")

In [443]:
def run_tests_optimized(use_real_bedrock: bool = True, categories: List[str] = None, model_id: str = None):
    """
    Función principal optimizada para prompt caching
    FASE 1: Todas las clasificaciones
    FASE 2: Todas las extracciones (agrupadas por categoría)
    """
    if categories is None:
        categories = CATEGORIES_ORDER
    
    # If model_id is specified, temporarily override environment variable
    original_model = None
    if model_id:
        original_model = os.environ.get("BEDROCK_MODEL")
        os.environ["BEDROCK_MODEL"] = model_id
        print(f"🤖 Using specified model: {model_id}")
        
        if is_anthropic_model(model_id):
            print(f"📡 Will use InvokeModel API for Anthropic model")
        elif is_nova_model(model_id):
            print(f"📡 Will use Converse API for Nova model")
        else:
            print(f"📡 Will use Converse API")
    
    print(f"🎯 Executing OPTIMIZED tests for categories: {categories}")
    print(f"🔧 Using real Bedrock: {'Yes' if use_real_bedrock else 'No (simulation)'}")
    print("🏗️  Flujo optimizado para prompt caching:")
    print("   1️⃣ FASE 1: Clasificar TODOS los documentos")
    print("   2️⃣ FASE 2: Extraer TODOS los documentos (agrupados por categoría)")
    
    try:
        # Load documents
        loader = DocumentLoader(TEST_DOCUMENTS_ROOT)
        loader.show_structure_summary()
        all_docs = loader.load_all_documents_ordered(categories)

        if not all_docs:
            print("❌ No se encontraron documentos de prueba.")
            print(f"   Coloca PDFs en: {TEST_DOCUMENTS_ROOT}/[CATEGORIA]/")
            return
        
        # Configure tester and reporter
        tester = DocumentTesterRefactored(PROJECT_STRUCTURE_ROOT, use_real_bedrock, model_id)
        results_path = PROJECT_STRUCTURE_ROOT / "results"
        reporter = ReportGeneratorOptimized(results_path)

        # Preparar lista de todos los documentos en orden
        all_test_docs = []
        for category in categories:
            if category in all_docs:
                all_test_docs.extend(all_docs[category])
        
        total_docs = len(all_test_docs)
        
        # Storage para resultados intermedios
        classification_results = {}  # doc_id -> classification_result
        
        print(f"\n🚀 INICIANDO PROCESO OPTIMIZADO ({total_docs} documentos)")
        print("="*70)
        
        # ================================
        # FASE 1: CLASIFICACIONES
        # ================================
        print(f"\n1️⃣ FASE 1: CLASIFICACIÓN DE TODOS LOS DOCUMENTOS")
        print("-" * 50)
        
        for i, test_doc in enumerate(all_test_docs, 1):
            print(f"🔍 [{i}/{total_docs}] Clasificando: {test_doc.category}/{test_doc.document_number}/{test_doc.name}")
            
            success, result = tester.classify_document(test_doc)
            
            # Crear un ID único para el documento
            doc_id = f"{test_doc.category}_{test_doc.document_number}_{test_doc.name}"
            classification_results[doc_id] = {
                "test_doc": test_doc,
                "success": success,
                "result": result
            }
            
            # Guardar resultado de clasificación inmediatamente
            reporter.save_classification_result(result, test_doc)
            
            status = "✅ PASS" if success else "❌ FAIL"
            classified_category = result.get("classification_data", {}).get("category", "UNKNOWN") if success else "ERROR"
            print(f"  {status} - Clasificado como: {classified_category}")
        
        print(f"\n✅ FASE 1 COMPLETADA: {len(classification_results)} documentos clasificados")
        
        # ================================
        # FASE 2: EXTRACCIONES
        # ================================
        print(f"\n2️⃣ FASE 2: EXTRACCIÓN DE TODOS LOS DOCUMENTOS")
        print("-" * 50)
        
        # Agrupar documentos por categoría clasificada para optimizar caching
        docs_by_classified_category = {}
        for doc_id, class_info in classification_results.items():
            if class_info["success"]:
                classified_category = class_info["result"].get("classification_data", {}).get("category", "UNKNOWN")
                if classified_category not in docs_by_classified_category:
                    docs_by_classified_category[classified_category] = []
                docs_by_classified_category[classified_category].append(doc_id)
        
        extraction_results = {}
        extraction_count = 0
        
        # Procesar extracciones agrupadas por categoría
        for category in CATEGORIES_ORDER + ["UNKNOWN"]:  # Seguir el orden + cualquier categoría extra
            if category not in docs_by_classified_category:
                continue
                
            category_docs = docs_by_classified_category[category]
            print(f"\n🔧 Procesando extracciones para categoría: {category} ({len(category_docs)} documentos)")
            
            for doc_id in category_docs:
                extraction_count += 1
                class_info = classification_results[doc_id]
                test_doc = class_info["test_doc"]
                classification_data = class_info["result"].get("classification_data", {})
                
                print(f"🔧 [{extraction_count}/{total_docs}] Extrayendo: {test_doc.category}/{test_doc.document_number}/{test_doc.name}")
                
                success, result = tester.extract_from_document(test_doc, classification_data)
                extraction_results[doc_id] = {
                    "success": success,
                    "result": result
                }
                
                # Guardar resultado de extracción inmediatamente
                reporter.save_extraction_result(result, test_doc, category)
                
                status = "✅ PASS" if success else "❌ FAIL"
                if result.get("skipped"):
                    status = "⏭️ SKIP"
                print(f"  {status}")
        
        print(f"\n✅ FASE 2 COMPLETADA: {len(extraction_results)} documentos procesados")
        
        # ================================
        # COMPILACIÓN DE RESULTADOS FINALES
        # ================================
        print(f"\n📊 COMPILANDO RESULTADOS FINALES")
        print("-" * 50)
        
        final_results = []
        for doc_id in classification_results.keys():
            class_info = classification_results[doc_id]
            test_doc = class_info["test_doc"]
            
            # Compilar resultado final
            final_result = {
                "document_name": test_doc.name,
                "category": test_doc.category,
                "document_number": test_doc.document_number,
                "classification_success": class_info["success"],
                "extraction_success": extraction_results.get(doc_id, {}).get("success", False),
                "classification_result": class_info["result"],
                "timestamp": datetime.now().isoformat()
            }
            
            # Agregar resultado de extracción si existe
            if doc_id in extraction_results:
                final_result["extraction_result"] = extraction_results[doc_id]["result"]
                final_result["extraction_success"] = extraction_results[doc_id]["success"]
            
            # Determinar éxito general
            final_result["overall_success"] = (
                final_result["classification_success"] and 
                final_result.get("extraction_success", False)
            )
            
            final_results.append(final_result)
        
        # Generar resumen final
        summary = reporter.save_final_summary(final_results)
        reporter.print_summary(summary)
        
        print("\n📁 ESTRUCTURA DE RESULTADOS:")
        print("   results/")
        print("   ├── classification/")
        print("   │   └── [doc_number]/")
        print("   │       └── classification_[category]_[filename]_[timestamp].json")
        print("   ├── extraction/")
        print("   │   └── [doc_number]/")
        print("   │       └── extraction_[category]_[filename]_[timestamp].json")
        print("   └── testing_summary_optimized_[timestamp].json")
        
        print(f"\n🎉 PROCESO COMPLETADO")
        print(f"📈 Estadísticas:")
        print(f"   • Documentos procesados: {total_docs}")
        print(f"   • Clasificaciones exitosas: {sum(1 for r in final_results if r['classification_success'])}")
        print(f"   • Extracciones exitosas: {sum(1 for r in final_results if r.get('extraction_success', False))}")
        print(f"   • Éxito general: {sum(1 for r in final_results if r['overall_success'])}")
        
        return summary
        
    finally:
        # Restore original environment variable
        if original_model is not None:
            if original_model:
                os.environ["BEDROCK_MODEL"] = original_model
            else:
                os.environ.pop("BEDROCK_MODEL", None)

In [446]:
summary = run_tests_optimized(use_real_bedrock=True, model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",categories=['RUT'])
#summary = run_tests_optimized(use_real_bedrock=True, model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",categories=['CERL'])

2025-09-02 16:33:15,058 - __main__ - INFO - 🔍 Buscando subcarpetas numeradas en: c:\Users\Usuario\Desktop\parservicios\bedrock-agent-par-servicios\testing\test_documents\RUT
2025-09-02 16:33:15,065 - __main__ - INFO - ✅ Encontradas 30 subcarpetas numeradas en RUT
2025-09-02 16:33:15,066 - __main__ - INFO - 📂 Procesando subcarpeta: RUT/08689024000292
2025-09-02 16:33:15,068 - __main__ - INFO - ✅ Encontrados 1 PDFs en RUT/08689024000292
2025-09-02 16:33:15,069 - __main__ - INFO -     PDF 1/1: 816_2020-02-29.pdf
2025-09-02 16:33:15,084 - __main__ - INFO - 📄 Cargado: 816_2020-02-29.pdf (127895 bytes)
2025-09-02 16:33:15,085 - __main__ - INFO - 📂 Procesando subcarpeta: RUT/31158722566
2025-09-02 16:33:15,087 - __main__ - INFO - ✅ Encontrados 1 PDFs en RUT/31158722566
2025-09-02 16:33:15,088 - __main__ - INFO -     PDF 1/1: 685_2020-02-29.pdf
2025-09-02 16:33:15,110 - __main__ - INFO - 📄 Cargado: 685_2020-02-29.pdf (639985 bytes)
2025-09-02 16:33:15,111 - __main__ - INFO - 📂 Procesando subca

🤖 Using specified model: us.anthropic.claude-sonnet-4-20250514-v1:0
📡 Will use InvokeModel API for Anthropic model
🎯 Executing OPTIMIZED tests for categories: ['RUT']
🔧 Using real Bedrock: Yes
🏗️  Flujo optimizado para prompt caching:
   1️⃣ FASE 1: Clasificar TODOS los documentos
   2️⃣ FASE 2: Extraer TODOS los documentos (agrupados por categoría)

📊 RESUMEN DE ESTRUCTURA:
✅ ACC: 28 subcarpetas, 28 con PDFs
✅ CECRL: 31 subcarpetas, 39 con PDFs
✅ CERL: 31 subcarpetas, 31 con PDFs
✅ RUB: 25 subcarpetas, 25 con PDFs
✅ RUT: 30 subcarpetas, 30 con PDFs

📈 TOTAL: 145 subcarpetas, 153 documentos procesables


2025-09-02 16:33:15,200 - __main__ - INFO - 📄 Cargado: 939_2020-02-29.pdf (176369 bytes)
2025-09-02 16:33:15,201 - __main__ - INFO - 📂 Procesando subcarpeta: RUT/800003675
2025-09-02 16:33:15,203 - __main__ - INFO - ✅ Encontrados 1 PDFs en RUT/800003675
2025-09-02 16:33:15,204 - __main__ - INFO -     PDF 1/1: 940_2020-02-29.pdf
2025-09-02 16:33:15,221 - __main__ - INFO - 📄 Cargado: 940_2020-02-29.pdf (194386 bytes)
2025-09-02 16:33:15,222 - __main__ - INFO - 📂 Procesando subcarpeta: RUT/800003781
2025-09-02 16:33:15,225 - __main__ - INFO - ✅ Encontrados 1 PDFs en RUT/800003781
2025-09-02 16:33:15,226 - __main__ - INFO -     PDF 1/1: 941_2020-02-29.pdf
2025-09-02 16:33:15,249 - __main__ - INFO - 📄 Cargado: 941_2020-02-29.pdf (809360 bytes)
2025-09-02 16:33:15,251 - __main__ - INFO - 📂 Procesando subcarpeta: RUT/800003837
2025-09-02 16:33:15,253 - __main__ - INFO - ✅ Encontrados 1 PDFs en RUT/800003837
2025-09-02 16:33:15,255 - __main__ - INFO -     PDF 1/1: 942_2020-02-29.pdf
2025-09-02


🚀 INICIANDO PROCESO OPTIMIZADO (30 documentos)

1️⃣ FASE 1: CLASIFICACIÓN DE TODOS LOS DOCUMENTOS
--------------------------------------------------
🔍 [1/30] Clasificando: RUT/08689024000292/816_2020-02-29.pdf


2025-09-02 16:33:43,403 - __main__ - INFO - Successfully extracted text content (3080 characters)
2025-09-02 16:33:43,410 - __main__ - INFO - 💾 Clasificación guardada: 08689024000292/classification_RUT_816_2020-02-29.pdf_20250902_163343.json
2025-09-02 16:33:43,410 - __main__ - INFO - 🔍 Clasificando: RUT/31158722566/685_2020-02-29.pdf
2025-09-02 16:33:43,410 - __main__ - INFO - 💾 Classification prompts saved: RUT_31158722566_20250902_163343
2025-09-02 16:33:43,425 - __main__ - INFO - 📄 PDF size for Anthropic model: 60928 bytes
2025-09-02 16:33:43,426 - __main__ - INFO - Calling InvokeModel API for model: us.anthropic.claude-sonnet-4-20250514-v1:0


...

  ✅ PASS
🔧 [28/30] Extrayendo: RUT/900686567/107_2020-02-29.pdf


2025-09-02 17:03:30,133 - __main__ - INFO - Successfully extracted text content (447 characters)
2025-09-02 17:03:30,138 - __main__ - INFO - 💾 Extracción guardada: 900686567/extraction_RUT_107_2020-02-29.pdf_20250902_170330.json
2025-09-02 17:03:30,141 - __main__ - INFO - 🔧 Extrayendo: RUT/901022555/7_2020-02-29.pdf → RUT
2025-09-02 17:03:30,144 - __main__ - INFO - 💾 Extraction prompts saved: RUT_20250902_170330
2025-09-02 17:03:30,148 - __main__ - INFO - 📄 PDF size for Anthropic model: 882545 bytes
2025-09-02 17:03:30,149 - __main__ - INFO - Calling InvokeModel API for model: us.anthropic.claude-sonnet-4-20250514-v1:0


  ✅ PASS
🔧 [29/30] Extrayendo: RUT/901022555/7_2020-02-29.pdf


2025-09-02 17:03:48,557 - __main__ - INFO - Successfully extracted text content (464 characters)
2025-09-02 17:03:48,557 - __main__ - INFO - 💾 Extracción guardada: 901022555/extraction_RUT_7_2020-02-29.pdf_20250902_170348.json
2025-09-02 17:03:48,753 - __main__ - INFO - 📊 Resumen final guardado: testing_summary_optimized_20250902_170348.json


  ✅ PASS

✅ FASE 2 COMPLETADA: 29 documentos procesados

📊 COMPILANDO RESULTADOS FINALES
--------------------------------------------------

📊 RESUMEN DE TESTING - PAR SERVICIOS (OPTIMIZADO PARA CACHING)
Total de pruebas: 30
Tasa de éxito en clasificación: 96.7%
Tasa de éxito en extracción: 96.7%
Tasa de éxito general: 96.7%
Modo de procesamiento: optimized_for_prompt_caching

📋 Resultados por categoría:
  RUT: 29/30 (96.7%)

🔍 Resultados detallados:
  ❌ FAIL - 816_2020-02-29.pdf (RUT)
  ✅ PASS - 685_2020-02-29.pdf (RUT)
  ✅ PASS - 935_2020-02-29.pdf (RUT)
  ✅ PASS - 936_2020-02-29.pdf (RUT)
  ✅ PASS - 937_2020-02-29.pdf (RUT)
  ✅ PASS - 938_2020-02-29.pdf (RUT)
  ✅ PASS - 939_2020-02-29.pdf (RUT)
  ✅ PASS - 940_2020-02-29.pdf (RUT)
  ✅ PASS - 941_2020-02-29.pdf (RUT)
  ✅ PASS - 942_2020-02-29.pdf (RUT)
  ✅ PASS - 943_2020-02-29.pdf (RUT)
  ✅ PASS - 945_2020-02-29.pdf (RUT)
  ✅ PASS - 947_2020-02-29.pdf (RUT)
  ✅ PASS - fQLjs9Kfe0ZNSOCr.pdf (RUT)
  ✅ PASS - RUT_TERPEL_16_02_2023___2_.p